## Building a text matching algorithm for question retrieval

In [2]:
import csv
import nltk as nltk
import pandas as pd
import numpy as np
from collections import defaultdict
import heapq
import itertools
from collections import Counter
from sklearn.decomposition import TruncatedSVD

In [3]:
%%time
# Load the dataset
dataset = pd.read_csv('data.tsv', sep='\t', error_bad_lines=False, quoting=csv.QUOTE_NONE)
print('Initial data size: ', dataset.shape)
dataset.dropna(inplace=True)
print('After dropping NAN values: ', dataset.shape)

Initial data size:  (363870, 6)
After dropping NAN values:  (363846, 6)
CPU times: user 1.39 s, sys: 166 ms, total: 1.55 s
Wall time: 1.56 s


### Text Preprocessing

In [4]:
import re
from string import digits
from string import punctuation

remove_digits = str.maketrans('', '', digits)
punct = (punctuation + 'ÔøΩüčé')

def preprocessing(doc):
    # remove punctuation and special characters
    translation = dict.fromkeys(map(ord, punct), None)  # Dictionary with punctuation to be removed
    doc = doc.lower().translate(translation)

    # remove non-english words
    doc = re.sub('([^\x00-\x7F])+', '', doc)

    # remove numbers
    # doc = re.sub('[0-9]+', '', doc)

    # tokenization
    tokens = nltk.word_tokenize(doc)

    # remove token that are considered as stop words
    stops = set(nltk.corpus.stopwords.words("english"))
    doc = [token for token in tokens if not token in stops]


    # lemmatization
    doc = [nltk.stem.WordNetLemmatizer().lemmatize(token) for token in doc]
    # doc = [nltk.stem.WordNetLemmatizer().lemmatize(token) for token in tokens]

    return doc


In [5]:
%%time

# Apply text preprocessing techniques for question 1 and question 2 documents
dataset['q2_tokens'] = dataset['question2'].apply(preprocessing)
dataset.head()

# select q1 100 questions with is_duplicate is 1
q1_data = dataset.loc[dataset['is_duplicate'] == 1]
q1_100 = q1_data.head(100)
q1_100['q1_tokens'] = q1_data['question1'].apply(preprocessing)
q1_100.head(10)

CPU times: user 3min 10s, sys: 32.4 s, total: 3min 42s
Wall time: 3min 58s


<timed exec>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,qid1,qid2,question1,question2,is_duplicate,q2_tokens,q1_tokens
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0,"[control, horniness]","[control, horny, emotion]"
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0,[mbbs],"[one, mbbs]"
7,106969,147570,787.0,What is the best self help book you have read?...,What are the top self help books I should read?,1.0,"[top, self, help, book, read]","[best, self, help, book, read, change, life]"
11,233239,71243,177376.0,What will be Hillary Clinton's policy towards ...,What will be Hilary Clinton's policy towards I...,1.0,"[hilary, clinton, policy, towards, india, beco...","[hillary, clinton, policy, towards, india, bec..."
13,11568,22332,22333.0,Which is the best book to study TENSOR for gen...,Which is the best book for tensor calculus?,1.0,"[best, book, tensor, calculus]","[best, book, study, tensor, general, relativit..."
16,54145,95620,95621.0,What are the coolest Android hacks and tricks ...,What are some cool hacks for Android phones?,1.0,"[cool, hack, android, phone]","[coolest, android, hack, trick, know]"
18,221213,289541,328497.0,Which are the best motivational videos?,What are some of the best motivational clips?,1.0,"[best, motivational, clip]","[best, motivational, video]"
19,85734,9827,9171.0,How do I lose weight fast?,What is the best way to reduce weight fast?,1.0,"[best, way, reduce, weight, fast]","[lose, weight, fast]"
24,33995,62359,62360.0,How does an IQ test work and what is determine...,How does IQ test works?,1.0,"[iq, test, work]","[iq, test, work, determined, iq, test]"
25,244506,357159,357160.0,Is it safe to use Xiaomi mobile phones?,Is it safe or unsafe to use Xiaomi Products?,1.0,"[safe, unsafe, use, xiaomi, product]","[safe, use, xiaomi, mobile, phone]"


### Build Inverted Index with TF-IDF and Text matching

In [6]:
def get_tfidf_inv_idx(data):
    word_doc_tf = defaultdict(list)
    inverted_index = defaultdict(list)
    for index, row in data.iterrows():
        # bag of words
        bow = dict(zip(*np.unique(row['q2_tokens'], return_counts=True)))

        # calculate term frequency
        for word in bow:
            tf = np.log10(1 + bow.get(word))
            word_doc_tf[word] += [(row['id'], tf)]

    # total number of documents
    N = data.shape[0]
    for word, docs in word_doc_tf.items():
        # calculate inverted document frequency
        idf = np.log10(N / len(docs))
        # idf = np.log(float(N)/len(docs) + 1)
        for doc_id, tf in docs:
            # calculate tfidf score and build inverted index
            inverted_index[word] += [(doc_id, tf * idf)]

    return inverted_index

In [7]:
def text_matching(data, doc, inv_idx, rank_num):
    doc_score = defaultdict(float)
    for word in doc:
        if word not in inv_idx: continue
        w_docs = inv_idx[word]

        for doc_id, tfidf in w_docs:
            doc_score[doc_id] += tfidf

    '''
    Document ranking
    retrieve top 50 matches
    '''
    top_docs = heapq.nlargest(50, doc_score, key=doc_score.get)

    # remove duplicates and select the question based on top ranks
    text_match = []
    for doc_id in top_docs:
        row = data.loc[data['id'] == doc_id]
        question = row['question2'].item()

        if question not in text_match:
            text_match += [question]
            if len(text_match) == rank_num: break

    return text_match

In [8]:
def probability(query, results):
    # Check the probability of top 2 and top 5
    top2_accuracy = 0
    top5_accuracy = 0
    query_len = len(query)

    for q_id in range(query_len):
        res = results[q_id]

        res_top2 = res[0:1]
        res_top5 = res[0:4]

        if query[q_id] in res_top2: top2_accuracy += 1
        if query[q_id] in res_top5: top5_accuracy += 1

    print('Top 2 Accuracy Rate = %.2f' % ((query_len - (query_len - top2_accuracy))/ query_len))
    print('Top 5 Accuracy Rate = %.2f' % ((query_len - (query_len - top5_accuracy))/ query_len))


In [9]:
%%time
# build the inverted index with tf-idf
inv_idx = get_tfidf_inv_idx(dataset)

# retrieve top 5 questions
text_match = []
for doc in q1_100['q1_tokens']:
    text_match += [text_matching(dataset, doc, inv_idx, 5)]

CPU times: user 2min 10s, sys: 2.9 s, total: 2min 13s
Wall time: 2min 35s


In [10]:
%%time
query = q1_100['question1'].tolist()
match_result = {'Query': query, 'Result': text_match}
result_df = pd.DataFrame(match_result)

CPU times: user 1.96 ms, sys: 630 µs, total: 2.59 ms
Wall time: 6.48 ms


In [11]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## display 5 queries out of 100
result_df.head(5)

,Query,Result
0,How do I control my horny emotions?,"[What is an effective way to control emotion and how do you release your negative emotion?, Do emotions impede logic or do emotions contribute to being rational? Are emotions logical to have?, How do I control my emotions and anger?, How can I control my emotions?, How do I control emotions and reactions in nervousness?]"
1,What can one do after MBBS?,"[I am first year MBBS student(ST) in one of new Aiims.I feel inferior to others due to my rank in entrance exam.can I get good results in MBBS exams?, Is it true that in order to get admission to usa after mbbs in india, ur medical college should be aiims only or u enter only on the basis of usmle? Basically does being from a good medical college in india guarantee you admission in usa or canada or some other country after mbbs or does it help?, How can I transfer from my MBBS medical college to another in Maharashtra after the 1st year of the MBBS?, Should I drop one more year to prepare for an MBBS?, What pay can one get after finishing mbbs in India on working abroad?]"
2,What is the best self help book you have read? Why? How did it change your life?,"[What is the best self help book you have read? Why? How did it change your life?, Which is the best self help book you've ever read?, What are the top self help books I should read?, What are some of the best non fiction books to read that can change one's perspective of life?, Philosophy of Mind: Reading self help books and upvoting 'life advice' questions in Quora. Are both of them mean the same thing?]"
3,What will be Hillary Clinton's policy towards India if she becomes president?,"[What will be Hillary Clinton's policy towards India if she becomes president?, If Hillary Clinton becomes the President, what will be the policy towards India?, What will be Hillary clintons India policy if she becomes president?, What will be Hillary Clinton foreign policy with respect to India once she becomes president?, What will be Hillary Clinton's policy for INDIA if she becomes the president?]"
4,Which is the best book to study TENSOR for general relativity from basic?,"[Which is the best book to study TENSOR for general relativity from basic?, What is the best book for self-learning Special and General Theory of Relativity. What are some beginner and advanced level books on these topics?, What is the best book on special relativity, Wolfgang Rindler's 'Introduction to Special relativity' or A.P. French's 'Special relativity'?, Which is the best book for special and general relativity with mathematics?, ""What is the difference between a """"data matrix"""" and a matrix (matrix operator), or a """"data tensor"""" and a tensor (tensor operator)?""]"


In [12]:
# Check probability of tf-idf ground-truth for top 2 and 5 results
probability(query, text_match)

Top 2 Accuracy Rate = 0.31
Top 5 Accuracy Rate = 0.39


### Text matching with sentence embedding by averaging word embedding

In [13]:
question2 = dataset['question2'].tolist()
question1 = q1_100['question1'].tolist()
q2_tokens = dataset['q2_tokens'].tolist()
q1_tokens = q1_100['q1_tokens'].tolist()
dim = 300

In [14]:

#load GloVe data model
def load_glove():
    filename = 'glove.6B/glove.6B.300d.txt'
    glove_data = defaultdict()
    file = open(filename, "r")

    for line in file:
        reader = line.split()
        word = reader[0]
        glove_data[word] = np.array([float(val) for val in reader[1:]])
    return glove_data

In [15]:
def get_sentence_embedding(docs, dim):
    sentence_embedding = np.zeros([len(docs), dim])
    glove_corpus = set(glove_data.keys())

    for index, doc in enumerate(docs):
        word_embedding = np.zeros([1, dim])
        word_embedding_count = 0
        average = 0
        for word in doc:
            # sum the word weight retrieved from glove and add to word embedding
            if word in glove_corpus:
                word_embedding += glove_data[word]
                word_embedding_count += 1

        # calculate average
        if word_embedding_count > 0:
            average = word_embedding / word_embedding_count
        else: average = word_embedding_count

        sentence_embedding[index, :] = average
    return sentence_embedding


In [16]:
def sentence_text_matching(q1_s_embedding, q2_s_embedding, rank_num):
    cos_sim = np.dot(q2_s_embedding, q1_s_embedding.reshape([-1, 1])) / np.linalg.norm(q1_s_embedding)
    cos_sim = cos_sim.flatten()
    cos_sim = cos_sim / np.linalg.norm(q2_s_embedding, axis=1)
    cos_sim[np.isnan(cos_sim)] = -1

    '''
    Document ranking
    retrieve top 50 matches
    '''
    idx = np.argpartition(cos_sim, -50)[-50:]
    top_doc = idx[np.argsort(cos_sim[idx])[::-1]]

    # remove duplicates and select the question based on top ranks
    text_match = []
    for doc_id in top_doc:
        question = question2[doc_id]
        if question not in text_match:
            text_match += [question]
            if len(text_match) == rank_num: break
    return text_match

In [17]:
%%time
## load glove dataset
glove_data = load_glove()

CPU times: user 41.5 s, sys: 2.67 s, total: 44.2 s
Wall time: 51.7 s


In [18]:
%%time
# retrieve sentence embedding for q1 and q2
q1_sentence_embedding = get_sentence_embedding(q1_tokens, dim)
q2_sentence_embedding = get_sentence_embedding(q2_tokens, dim)

## retrieve top 5 results
text_match = []
for query in q1_sentence_embedding:
    text_match += [sentence_text_matching(query, q2_sentence_embedding, 5)]
match_result = {'Query': question1, 'Result': text_match}
e_result_df = pd.DataFrame(match_result)
e_result_df.head()

<ipython-input-16-a81221f1d529>:4: RuntimeWarning: invalid value encountered in true_divide
  cos_sim = cos_sim / np.linalg.norm(q2_s_embedding, axis=1)


CPU times: user 1min 13s, sys: 42.5 s, total: 1min 55s
Wall time: 1min 46s


,Query,Result
0,How do I control my horny emotions?,"[How do we control our emotions?, How can I control my emotions?, How should I control my emotion?, How can I control emotions?, How do I have control over my emotions?]"
1,What can one do after MBBS?,"[What can I do after my MBBs?, What do i do after my MBBS ?, What have to Do after MBBS?, Is M.B.B.S a good major?, Which course should I choose after an MBBS?]"
2,What is the best self help book you have read? Why? How did it change your life?,"[What is the best self help book you have read? Why? How did it change your life?, Which is the best self help book you've ever read?, Can you name a few books (not the self help books) that have had a great impact on you and changed you for the better?, What is the one book that you think reading it will change my life to the better immediately?, What are the top self help books I should read?]"
3,What will be Hillary Clinton's policy towards India if she becomes president?,"[If Hillary Clinton becomes the President, what will be the policy towards India?, What will be Hillary Clinton's policy towards India if she becomes president?, What will be Hillary Clinton's policy for INDIA if she becomes the president?, What will be Hillary clintons India policy if she becomes president?, What will be Hilary Clinton's policy towards India if she become President?]"
4,Which is the best book to study TENSOR for general relativity from basic?,"[Which is the best book to study TENSOR for general relativity from basic?, Which is the best book for special and general relativity with mathematics?, What is the best book for self-learning Special and General Theory of Relativity. What are some beginner and advanced level books on these topics?, Which is the best book to understand tensors?, Which are the best books on relativity?]"


In [19]:
%%time
# Check probability of text matching of sentence embedding on ground-truth for top 2 and 5 results
probability(question1, text_match)


Top 2 Accuracy Rate = 0.36
Top 5 Accuracy Rate = 0.42
CPU times: user 1.22 ms, sys: 1.35 ms, total: 2.58 ms
Wall time: 2.07 ms


### A Simple but tough-to-beat Baseline for Sentence Embeddings
1. Retrieve the frequency down weighted sentence embedding <br/>
2. Build an svd
3. Calculate the svd
$v_s \leftarrow v_s - uu^Tv_s$

In [20]:
# Calculate P(w)
# Calculate the word frequency for question
def get_word_probability(docs):
    # Question2 vocabulary
    vocab = list(itertools.chain.from_iterable(docs))
    P = Counter(vocab)
    q2_vocab_len = len(vocab)

    # calculate the probability of each word P(w)
    for idx, word in  P.items():
        P[idx] = word/q2_vocab_len

    return P

In [21]:
def get_freq_down_weighted_sentence_embeddings(docs, dim):
    # retrieve the word probability
    P = get_word_probability(docs)

    # initialise sentence embedding
    sentence_embedding = np.zeros([len(docs), dim])
    glove_corpus = set(glove_data.keys())
    a = 0.0005

    # calculate the word embedding using glove and P(w)
    for index, doc in enumerate(docs):
        word_embedding = np.zeros([1, dim])
        word_embedding_count = 0

        for word in doc:
            # sum the word weight retrieved from glove and add to word embedding
            if word in glove_corpus:
                word_embedding += glove_data[word] * (a/(a + P[word]))
                word_embedding_count += 1

        # calculate average
        if word_embedding_count > 0:
            average = word_embedding / word_embedding_count
        else: average = word_embedding_count

        sentence_embedding[index, :] = average
    return sentence_embedding

In [22]:
def tb_sentence_text_matching(q1_s_embedding, tb_q2_s_embedding,  u, rank_num):
    tb_q1_s_embedding =  q1_s_embedding - np.dot(q1_s_embedding, np.outer(u, u))
    cos_sim = np.dot(tb_q2_s_embedding, tb_q1_s_embedding.reshape([-1, 1])) / np.linalg.norm(tb_q1_s_embedding)
    cos_sim = cos_sim.flatten()
    cos_sim = cos_sim / np.linalg.norm(tb_q2_s_embedding, axis=1)
    cos_sim[np.isnan(cos_sim)] = -1

    '''
    Document ranking
    retrieve top 50 matches
    '''
    idx = np.argpartition(cos_sim, -50)[-50:]
    top_doc = idx[np.argsort(cos_sim[idx])[::-1]]

    # remove duplicates and select the question based on top ranks
    text_match = []
    for doc_id in top_doc:
        question = question2[doc_id]
        if question not in text_match:
            text_match += [question]
            if len(text_match) == rank_num: break
    return text_match

In [23]:
%%time
# calculate the frequency down weighted sentence embeddings for q1 and q2
fdw_q1_sentence_embedding = get_freq_down_weighted_sentence_embeddings(q1_tokens, dim)
fdw_q2_sentence_embedding = get_freq_down_weighted_sentence_embeddings(q2_tokens, dim)

#build svd
svd = TruncatedSVD(n_components=1, n_iter=7, random_state=42).fit(fdw_q2_sentence_embedding)
u = svd.components_
tb_q2_sentence_embedding = fdw_q2_sentence_embedding - np.dot(fdw_q2_sentence_embedding, np.outer(u, u))

# sentence text matching using the tb sentence embedding
## retrieve top 5 results
text_match = []
for query in fdw_q1_sentence_embedding:
    text_match += [tb_sentence_text_matching(query, tb_q2_sentence_embedding, u, 5)]
match_result = {'Query': question1, 'Result': text_match}
tb_result_df = pd.DataFrame(match_result)
tb_result_df.head()


<ipython-input-22-dc9f83cfb77b>:5: RuntimeWarning: invalid value encountered in true_divide
  cos_sim = cos_sim / np.linalg.norm(tb_q2_s_embedding, axis=1)


CPU times: user 1min 32s, sys: 41.3 s, total: 2min 13s
Wall time: 2min 2s


,Query,Result
0,How do I control my horny emotions?,"[What is the best way to control our emotions?, How can I control my emotions?, How do we control our emotions?, How should I control my emotion?, How do I have control over my emotions?]"
1,What can one do after MBBS?,"[What have to Do after MBBS?, What do i do after my MBBS ?, What can I do after my MBBs?, Is M.B.B.S a good major?, How should I study first year MBBS?]"
2,What is the best self help book you have read? Why? How did it change your life?,"[What is the best self help book you have read? Why? How did it change your life?, Which is the best self help book you've ever read?, Which books should I read now in self improvment?, What are the top self help books I should read?, Which books can help us for self study?]"
3,What will be Hillary Clinton's policy towards India if she becomes president?,"[If Hillary Clinton becomes the President, what will be the policy towards India?, What will be Hillary Clinton's policy towards India if she becomes president?, What will be Hillary clintons India policy if she becomes president?, What will be Hillary Clinton's policy for INDIA if she becomes the president?, What will be Hillary Clinton foreign policy with respect to India once she becomes president?]"
4,Which is the best book to study TENSOR for general relativity from basic?,"[Which is the best book to study TENSOR for general relativity from basic?, Which is the best book for special and general relativity with mathematics?, Is it possible that General relativity is an approximation?, What is the theory of relativity?, What are the basic differences among the theory of relativity and quantum theory?]"


In [24]:
%%time
# Check probability of text matching of sentence embedding on ground-truth for top 2 and 5 results
probability(question1, text_match)

Top 2 Accuracy Rate = 0.33
Top 5 Accuracy Rate = 0.41
CPU times: user 283 µs, sys: 216 µs, total: 499 µs
Wall time: 336 µs
